# Comparativo nacional y regiones foco RNP 2023



Notebook dedicado a contrastar el panorama nacional del sistema RNP 2023 con las regiones foco: Los Ríos, Biobío, Metropolitana de Santiago, Los Lagos y Araucanía.

## 1. Set Up Environment



Configuramos librerías base, opciones de visualización y constantes para asegurar reproducibilidad y estilo consistente.

In [1]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, Iterable

import math

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Markdown

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 40)
pd.set_option("display.float_format", "{:,.2f}".format)

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT != PROJECT_ROOT.parent:
    if (PROJECT_ROOT / "data").is_dir() and (PROJECT_ROOT / "src").is_dir():
        break
    PROJECT_ROOT = PROJECT_ROOT.parent
else:
    raise FileNotFoundError("No se encontró la raíz del proyecto con carpetas 'data' y 'src'.")

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

ECONOMIST_COLORS = [
    "#005f73",
    "#0a9396",
    "#94d2bd",
    "#ee9b00",
    "#ca6702",
    "#bb3e03",
    "#ae2012",
    "#001219",
]

FOCUS_REGIONS = [
    "Los Ríos",
    "Biobío",
    "Metropolitana De Santiago",
    "Los Lagos",
    "Araucanía",
]

## 2. Define Utility Functions



Funciones auxiliares para estilizar gráficos, formatear toneladas y construir tablas comparativas.

In [2]:
def apply_economist_style(

    fig: go.Figure,

    *,

    font_family: str = "Georgia, serif",

    base_color: str = "#355070",

    highlight_color: str = "#d62828",

    background_color: str = "#f7f2ea",

    grid_color: str = "#d9d3c5",

    underline: bool = True,

    auto_marker_line: bool = True,

) -> go.Figure:

    text_color = "#1e1b18"

    fig.update_layout(

        template="plotly_white",

        font=dict(family=font_family, color=text_color, size=15),

        title=dict(font=dict(family=font_family, size=22, color=text_color), x=0.0, y=0.96),

        xaxis=dict(

            title=dict(font=dict(family=font_family, color=text_color, size=14)),

            tickfont=dict(family=font_family, color=text_color, size=12),

            showgrid=True,

            gridcolor=grid_color,

            griddash="dot",

            zeroline=False,

            linecolor=grid_color,

        ),

        yaxis=dict(

            title=dict(font=dict(family=font_family, color=text_color, size=14)),

            tickfont=dict(family=font_family, color=text_color, size=12),

            showgrid=False,

            zeroline=False,

            linecolor=grid_color,

        ),

        legend=dict(font=dict(family=font_family, color=text_color, size=12)),

        hoverlabel=dict(bgcolor="#ffffff", font=dict(family=font_family, color=text_color)),

        bargap=0.28,

        plot_bgcolor=background_color,

        paper_bgcolor=background_color,

        margin=dict(l=90, r=40, t=95, b=70),

        colorway=[base_color],

    )



    if underline:

        underline_shape = dict(

            type="line",

            x0=0.0,

            x1=0.22,

            y0=1.09,

            y1=1.09,

            xref="paper",

            yref="paper",

            line=dict(color=highlight_color, width=4),

        )

        existing_shapes = list(fig.layout.shapes) if fig.layout.shapes else []

        fig.update_layout(shapes=[underline_shape, *existing_shapes])

    else:

        fig.update_layout(shapes=[])



    if auto_marker_line:

        fig.update_traces(

            marker_line_color=highlight_color,

            marker_line_width=0.7,

            selector=dict(type="bar"),

        )



    for ann in fig.layout.annotations or []:

        ann.font = dict(family=font_family, color="#4a4338", size=12)



    return fig





def humanize_tonnes(value: float) -> str:

    if value >= 1_000_000:

        return f"{value / 1_000_000:.1f} millones t"

    if value >= 1_000:

        return f"{value / 1_000:.1f} mil t"

    return f"{value:,.0f} t"





def format_share(value: float) -> str:

    return f"{value * 100:.1f}%"

## 3. Run Core Computations

Cargamos los datasets depurados, elaboramos métricas nacionales y construimos cuadros comparativos para las regiones foco.

> **Nota de estilo**: Para las visualizaciones clave adoptaremos la línea gráfica de [Our World in Data](https://github.com/owid/owid-grapher): tipografía sans-serif, fondos claros y acentos mínimos que destaquen comparaciones relevantes.

In [3]:
clean_df = pd.read_parquet(PROCESSED_DIR / "rinp_generacion_2023_cl_clean.parquet")



region_df = pd.read_csv(PROCESSED_DIR / "region_summary.csv", sep=";")

region_df.columns = ["region", "toneladas_sum", "toneladas_promedio", "registros"]

region_df["toneladas_sum"] = pd.to_numeric(region_df["toneladas_sum"], errors="coerce")

region_df["registros"] = pd.to_numeric(region_df["registros"], errors="coerce", downcast="integer")

region_df = region_df.sort_values("toneladas_sum", ascending=False).reset_index(drop=True)



treatment_df = pd.read_csv(PROCESSED_DIR / "region_treatment.csv", sep=";")

treatment_df.columns = ["region", "tratamiento_nivel_1", "toneladas"]

treatment_df["toneladas"] = pd.to_numeric(treatment_df["toneladas"], errors="coerce")



focus_region_df = region_df[region_df["region"].isin(FOCUS_REGIONS)].copy()

focus_region_df["region"] = pd.Categorical(focus_region_df["region"], categories=FOCUS_REGIONS, ordered=True)

focus_region_df = focus_region_df.sort_values("toneladas_sum", ascending=False).reset_index(drop=True)



overall_mix = (

    treatment_df.groupby("tratamiento_nivel_1", as_index=False)["toneladas"].sum()

    .sort_values("toneladas", ascending=False)

    .reset_index(drop=True)

)

overall_mix["share"] = overall_mix["toneladas"] / overall_mix["toneladas"].sum()



focus_treatment_share_df = treatment_df[treatment_df["region"].isin(FOCUS_REGIONS)].copy()

focus_treatment_share_df["share"] = focus_treatment_share_df["toneladas"] / focus_treatment_share_df.groupby("region")["toneladas"].transform("sum")

focus_treatment_share_df["region"] = pd.Categorical(focus_treatment_share_df["region"], categories=FOCUS_REGIONS, ordered=True)



los_rios_total = focus_region_df.loc[focus_region_df["region"] == "Los Ríos", "toneladas_sum"].sum()

los_rios_share = los_rios_total / region_df["toneladas_sum"].sum()



headline_metrics = pd.DataFrame(

    {

        "Indicador": [

            "Residuos totales registrados",

            "Regiones con reportes",

            "Establecimientos únicos",

            "Tratamiento predominante",

        ],

        "Valor": [

            humanize_tonnes(clean_df["cantidad_toneladas"].sum()),

            clean_df["region"].nunique(),

            clean_df["nombre_establecimiento"].nunique(),

            overall_mix.iloc[0]["tratamiento_nivel_1"],

        ],

    }

)



headline_metrics

,Indicador,Valor
0,Residuos totales registrados,10.4 millones t
1,Regiones con reportes,16
2,Establecimientos únicos,8411
3,Tratamiento predominante,Eliminación


## 4. Validate Outputs



Verificamos resultados mediante tablas resumidas, visualizaciones y aserciones ligeras que confirmen consistencia de los datos.

#### Diferencias entre regiones foco

Complementamos el panorama nacional con un zoom en la canasta de regiones que nos pidió el equipo técnico.

In [4]:
assert not clean_df.empty, "El dataset limpio no puede estar vacío"

assert set(FOCUS_REGIONS).issubset(set(region_df["region"])), "Faltan regiones foco en el resumen"



headline_table = go.Figure(

    data=[

        go.Table(

            columnorder=[1, 2],

            header=dict(

                values=["Indicador", "Valor"],

                fill_color="#1e293b",

                font=dict(color="#ffffff", family="Georgia, serif", size=16),

                align="left",

                height=38,

            ),

            cells=dict(

                values=[headline_metrics["Indicador"].tolist(), headline_metrics["Valor"].tolist()],

                fill_color=[["#faf6f0", "#f1e7db"] * 2],

                font=dict(color="#1e1b18", family="Georgia, serif", size=14),

                align="left",

                height=32,

            ),

        )

    ]

)

headline_table.update_layout(

    margin=dict(l=0, r=0, t=10, b=0),

    height=250,

    paper_bgcolor="#f7f2ea",

    annotations=[

        dict(

            text="Fuente: RNP 2023",

            x=0,

            y=-0.15,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=11, color="#6e6252"),

        )

    ],

)

headline_table.add_shape(

    type="line",

    x0=0.0,

    x1=0.22,

    y0=1.08,

    y1=1.08,

    xref="paper",

    yref="paper",

    line=dict(color="#d62828", width=4),

)

headline_table

In [5]:
national_bar_df = (

    region_df[["region", "toneladas_sum"]]

    .dropna()

    .sort_values("toneladas_sum", ascending=False)

    .reset_index(drop=True)

)



region_wrap_map = {

    "Metropolitana De Santiago": "Metropolitana\nde Santiago",

    "Los Lagos": "Los Lagos",

    "Los Ríos": "Los Ríos",

}

national_bar_df["region_display"] = (

    national_bar_df["region"].astype(str).map(region_wrap_map).fillna(national_bar_df["region"].astype(str))

)



total_tonnes = national_bar_df["toneladas_sum"].sum()

national_bar_df["tonnes_millions"] = national_bar_df["toneladas_sum"] / 1_000_000

national_bar_df["share"] = national_bar_df["toneladas_sum"] / total_tonnes



def format_spanish_number(number: float) -> str:

    formatted = f"{number:,.1f}"

    return formatted.replace(",", "X").replace(".", ",").replace("X", ".")



national_bar_df["label"] = national_bar_df.apply(

    lambda row: f"{format_spanish_number(row['tonnes_millions'])} M t · {format_spanish_number(row['share'] * 100)}%",

    axis=1,

)



highlight_color = "#d62828"

neutral_color = "#b5b8bf"

bar_colors = [highlight_color if region == "Los Ríos" else neutral_color for region in national_bar_df["region"]]



national_fig = go.Figure(

    go.Bar(

        x=national_bar_df["tonnes_millions"],

        y=national_bar_df["region_display"],

        orientation="h",

        marker=dict(color=bar_colors, line=dict(color="rgba(0,0,0,0)", width=0)),

        text=national_bar_df["label"],

        textposition="outside",

        textfont=dict(family="Georgia, serif", color="#1e1b18", size=12),

        hovertemplate="<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<br>Participación nacional: %{customdata[2]:.1%}<extra></extra>",

        customdata=national_bar_df[["region", "toneladas_sum", "share"]].to_numpy(),

        showlegend=False,

    )

)



max_millions = national_bar_df["tonnes_millions"].max()

tick_max = math.ceil((max_millions + 0.01) / 0.5) * 0.5

xaxis_range = [0, tick_max]



national_fig = apply_economist_style(

    national_fig,

    base_color=neutral_color,

    underline=False,

    auto_marker_line=False,

)



tick_vals = [round(0.5 * i, 1) for i in range(int(tick_max / 0.5) + 1)]

tick_text = [f"{format_spanish_number(val)} M" for val in tick_vals]



national_fig.update_traces(marker_line_color="rgba(0,0,0,0)")



national_fig.update_layout(

    title="Toneladas declaradas por región",

    xaxis_title="Toneladas",

    yaxis_title="",

    xaxis=dict(tickvals=tick_vals, ticktext=tick_text, range=xaxis_range, dtick=0.5),

    yaxis=dict(categoryorder="array", categoryarray=national_bar_df["region_display"].tolist()),

    width=1280,

    height=720,

    margin=dict(l=220, r=120, t=140, b=100),

    annotations=[

        dict(

            text="Chile, RNP 2023",

            x=0,

            y=1.05,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=14, color="#4a4338"),

        ),

        dict(

            text="Fuente: Registro Nacional de Emisiones y Transferencias de Residuos (RNP) 2023",

            x=0,

            y=-0.18,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=11, color="#6e6252"),

        ),

    ],

)



national_fig

#### Lollipop regional

Exploramos la distribución con un lollipop horizontal que facilita comparar órdenes y magnitudes en millones de toneladas.

In [6]:
import math

lollipop_df = (
    region_df[["region", "toneladas_sum"]]
    .dropna()
    .sort_values("toneladas_sum", ascending=False)
    .reset_index(drop=True)
)

region_wrap_map = {
    "Metropolitana De Santiago": "Metropolitana\nde Santiago",
    "Los Ríos": "Los Ríos",
    "Los Lagos": "Los Lagos",
}
lollipop_df["region_display"] = (
    lollipop_df["region"].astype(str).map(region_wrap_map).fillna(lollipop_df["region"].astype(str))
)

total_tonnes = lollipop_df["toneladas_sum"].sum()
lollipop_df["tonnes_millions"] = lollipop_df["toneladas_sum"] / 1_000_000
lollipop_df["participacion"] = lollipop_df["toneladas_sum"] / total_tonnes

def format_spanish(number: float) -> str:
    formatted = f"{number:,.1f}"
    return formatted.replace(",", "X").replace(".", ",").replace("X", ".")

lollipop_df["label"] = lollipop_df.apply(
    lambda row: f"{format_spanish(row['tonnes_millions'])} M t · {format_spanish(row['participacion'] * 100)}%",
    axis=1,
)

highlight_color = "#d62828"
neutral_color = "#9ca3af"

fig = go.Figure()

for _, row in lollipop_df.iterrows():
    color = highlight_color if row["region"] == "Los Ríos" else neutral_color
    fig.add_trace(
        go.Scatter(
            x=[0, row["tonnes_millions"]],
            y=[row["region_display"], row["region_display"]],
            mode="lines",
            line=dict(color=color, width=3),
            hoverinfo="skip",
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[row["tonnes_millions"]],
            y=[row["region_display"]],
            mode="markers+text",
            marker=dict(color=color, size=14, line=dict(color="#ffffff", width=1.5)),
            text=[row["label"]],
            textposition="middle right",
            textfont=dict(family="Georgia, serif", color="#1e1b18", size=12),
            hovertemplate=(
                "<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<br>Participación nacional: %{customdata[2]:.1%}<extra></extra>"
            ),
            customdata=[
                [
                    row["region"],
                    row["toneladas_sum"],
                    row["participacion"],
                ]
            ],
            showlegend=False,
        )
    )

fig = apply_economist_style(fig, base_color=neutral_color)

if fig.layout.shapes:
    fig.layout.shapes[0].update(x0=-0.22, x1=0.84)

max_millions = lollipop_df["tonnes_millions"].max()
tick_max = math.ceil(max_millions * 10) / 10
tick_step = max(0.2, tick_max / 6)
tick_values = [round(x, 1) for x in [i * tick_step for i in range(int(tick_max / tick_step) + 1)]]
tick_values = sorted(set(tick_values))
tick_text = [f"{format_spanish(val)} M" for val in tick_values]

x_upper_padding = tick_step if tick_step > 0 else 0.2
xaxis_range = [0, tick_max + x_upper_padding]

fig.update_traces(textfont=dict(size=12, family="Georgia, serif"))

fig.update_layout(
    title="Toneladas declaradas por región",
    title_font=dict(size=24),
    xaxis_title="Toneladas",
    yaxis_title="",
    xaxis=dict(
        tickvals=tick_values,
        ticktext=tick_text,
        rangemode="tozero",
        range=xaxis_range,
    ),
    yaxis=dict(
        categoryorder="array",
        categoryarray=lollipop_df["region_display"].tolist(),
    ),
    width=1380,
    height=760,
    margin=dict(l=200, r=170, t=130, b=130),
    annotations=[
        dict(
            text="Chile, RNP 2023",
            x=0,
            y=1.06,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(family="Georgia, serif", size=14, color="#4a4338"),
        ),
        dict(
            text="Fuente: Registro Nacional de Emisiones y Transferencias de Residuos (RNP) 2023",
            x=0,
            y=-0.22,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(family="Georgia, serif", size=11, color="#6e6252"),
        ),
    ],
)

fig

In [7]:
focus_plot = focus_region_df.sort_values("toneladas_sum", ascending=False).copy()


focus_plot["region"] = focus_plot["region"].astype(str)


region_label_map = {


    "Metropolitana": "Metropolitana",

    "Los Ríos": "Los Ríos",

    "Los Lagos": "Los Lagos",

}

focus_plot["region_display"] = focus_plot["region"].map(region_label_map).fillna(focus_plot["region"])

focus_plot["participacion"] = focus_plot["toneladas_sum"] / region_df["toneladas_sum"].sum()

focus_plot["tonnes_millions"] = focus_plot["toneladas_sum"] / 1_000_000



def format_spanish(number: float, decimals: int = 1) -> str:

    formatted = f"{number:,.{decimals}f}"

    return formatted.replace(",", "X").replace(".", ",").replace("X", ".")



focus_plot["label"] = focus_plot.apply(

    lambda row: f"{format_spanish(row['tonnes_millions'])} M t\n({format_spanish(row['participacion'] * 100)}%)",

    axis=1,

)



palette_focus = {

    "Los Ríos": "#d62828",

    "Biobío": "#495057",

    "Metropolitana De Santiago": "#686d76",

    "Los Lagos": "#8c8f99",

    "Araucanía": "#b0b4ba",

}

neutral_color = "#686d76"

bar_colors = [palette_focus.get(region, neutral_color) for region in focus_plot["region"]]



focus_fig = go.Figure(

    go.Bar(

        x=focus_plot["region_display"],

        y=focus_plot["tonnes_millions"],

        marker=dict(color=bar_colors, line=dict(color="rgba(0,0,0,0)", width=0)),

        text=focus_plot["label"],

        textposition="outside",

        textfont=dict(family="Georgia, serif", color="#1e1b18", size=13),

        hovertemplate="<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<br>Participación nacional: %{customdata[2]:.1%}<extra></extra>",

        customdata=focus_plot[["region", "toneladas_sum", "participacion"]].to_numpy(),

        cliponaxis=False,

        showlegend=False,

    )

)



focus_fig = apply_economist_style(

    focus_fig,

    base_color=neutral_color,

    underline=True,

    auto_marker_line=False,

)



if focus_fig.layout.shapes:

    focus_fig.layout.shapes[0].update(x0=-0.07, x1=0.95)



max_millions = focus_plot["tonnes_millions"].max()

tick_step = 0.25 if max_millions <= 2.0 else 0.5

tick_max = math.ceil((max_millions + tick_step * 0.2) / tick_step) * tick_step

tick_vals = [round(tick_step * i, 2) for i in range(int(tick_max / tick_step) + 1)]

tick_text = [f"{format_spanish(val)} M" for val in tick_vals]

yaxis_upper = tick_max + tick_step * 0.6



focus_fig.update_layout(

    title="Regiones foco: toneladas declaradas",

    xaxis_title="",

    yaxis_title="Millones de toneladas",

    xaxis=dict(

        tickvals=focus_plot["region_display"].tolist(),

        ticktext=focus_plot["region_display"].tolist(),

        tickangle=0,

        tickfont=dict(family="Georgia, serif", size=13, color="#1e1b18"),

    ),

    yaxis=dict(tickvals=tick_vals, ticktext=tick_text, range=[0, yaxis_upper]),

    bargap=0.35,

    width=1200,

    height=720,

    margin=dict(l=120, r=90, t=140, b=160),

    annotations=[

        dict(

            text="Chile, RNP 2023",

            x=0,

            y=1.05,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=14, color="#4a4338"),

        ),

        dict(

            text="Los Ríos se destaca en rojo para enfatizar su liderazgo entre las regiones foco",

            x=0,

            y=-0.2,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=11, color="#6e6252"),

        ),

        dict(

            text="Fuente: Registro Nacional de Emisiones y Transferencias de Residuos (RNP) 2023",

            x=0,

            y=-0.28,

            xref="paper",

            yref="paper",

            showarrow=False,

            font=dict(family="Georgia, serif", size=11, color="#6e6252"),

        ),

    ],

)

focus_fig

## 5. Save Artifacts

Guardamos tablas y visualizaciones clave para su uso en reportes y presentaciones del equipo.

In [8]:
artifacts_dir = PROJECT_ROOT / "artifacts" / "rnp_comparativo_regiones"

tables_dir = artifacts_dir / "tables"

figures_dir = artifacts_dir / "figures"

for directory in (tables_dir, figures_dir):

    directory.mkdir(parents=True, exist_ok=True)



headline_metrics.to_csv(tables_dir / "headline_metrics.csv", index=False)

focus_region_df.to_csv(tables_dir / "focus_regions_tonnes.csv", index=False)



national_fig.write_html(figures_dir / "national_tonnes.html", include_plotlyjs="cdn")

focus_fig.write_html(figures_dir / "focus_regions_tonnes.html", include_plotlyjs="cdn")



Markdown(

    f"""

Se exportaron artefactos a `{artifacts_dir.relative_to(PROJECT_ROOT)}`:



- Tablas: `{tables_dir.relative_to(PROJECT_ROOT)}`

- Gráficos interactivos: `{figures_dir.relative_to(PROJECT_ROOT)}`

"""

)



Se exportaron artefactos a `artifacts/rnp_comparativo_regiones`:



- Tablas: `artifacts/rnp_comparativo_regiones/tables`

- Gráficos interactivos: `artifacts/rnp_comparativo_regiones/figures`



In [21]:
import math

ECONOMIST_FONT = globals().get("ECONOMIST_FONT", "Georgia, serif")

treatment_focus_plot = focus_treatment_share_df.copy()
treatment_focus_plot["share_pct"] = treatment_focus_plot["share"] * 100
treatment_focus_plot = treatment_focus_plot.sort_values(
    ["region", "share_pct"], ascending=[True, False]
)

top_treatments = (
    overall_mix.sort_values("share", ascending=False)["tratamiento_nivel_1"].unique().tolist()
)

priority_labels = ["Eliminación", "Valorización", "Recolección"]
primary_treatments = [label for label in priority_labels if label in top_treatments]
for treatment in top_treatments:
    if treatment not in primary_treatments:
        primary_treatments.append(treatment)
primary_treatments = primary_treatments[:3]

dumbbell_base = (
    treatment_focus_plot[
        treatment_focus_plot["tratamiento_nivel_1"].isin(primary_treatments)
    ]
    .pivot_table(
        index="region",
        columns="tratamiento_nivel_1",
        values="share_pct",
        aggfunc="sum",
        observed=False,
    )
    .reindex(FOCUS_REGIONS)
)

connector_color = "#94a3b8"
background_color = "#f7f2ea"
grid_color = "#d9d3c5"
text_color = "#1e1b18"
highlight_color = "#d62828"

marker_colors = {
    primary_treatments[i]: ECONOMIST_COLORS[i]
    for i in range(min(len(primary_treatments), len(ECONOMIST_COLORS)))
}

dumbbell_fig = go.Figure()

connector_padding = 0.6  # keep connectors from overlapping markers

for region in dumbbell_base.index:
    row = dumbbell_base.loc[region] if region in dumbbell_base.index else None
    values = row.dropna() if row is not None else pd.Series(dtype=float)
    if values.empty:
        continue
    sorted_values = values.sort_values()
    previous_share = None
    for idx, (treatment, share_value) in enumerate(sorted_values.items()):
        color = marker_colors.get(
            treatment, ECONOMIST_COLORS[(idx + 2) % len(ECONOMIST_COLORS)]
        )
        hover_text = (
            f"<b>{region}</b><br>{treatment}: {share_value:.1f}%<extra></extra>"
        )
        show_legend = region == FOCUS_REGIONS[0]

        if previous_share is not None:
            direction = 1 if share_value >= previous_share else -1
            gap = abs(share_value - previous_share)
            padding = min(connector_padding, gap / 2)
            start = previous_share + direction * padding
            end = share_value - direction * padding
            if (direction == 1 and start < end) or (direction == -1 and start > end):
                dumbbell_fig.add_trace(
                    go.Scatter(
                        x=[start, end],
                        y=[region, region],
                        mode="lines",
                        line=dict(color=connector_color, width=2),
                        hoverinfo="skip",
                        showlegend=False,
                    )
                )

        dumbbell_fig.add_trace(
            go.Scatter(
                x=[share_value],
                y=[region],
                mode="markers+text",
                marker=dict(
                    color=color,
                    size=12,
                    line=dict(color="#ffffff", width=1.2)
                ),
                text=[f"{share_value:.0f}%"],
                textposition="bottom center",
                textfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
                customdata=[[region, treatment, share_value]],
                hovertemplate=hover_text,
                name=treatment,
                legendgroup=treatment,
                showlegend=show_legend,
            )
        )

        previous_share = share_value

underline_x0 = -0.20
underline_x1 = 0.88
underline_y = 1.12

dumbbell_fig.update_layout(
    template="plotly_white",
    title=dict(
        text="Tratamientos en regiones foco",
        font=dict(family=ECONOMIST_FONT, size=24, color=text_color),
        x=0.05,
        y=0.96,
    ),
    font=dict(family=ECONOMIST_FONT, size=15, color=text_color),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    margin=dict(l=220, r=160, t=120, b=80),
    hoverlabel=dict(bgcolor="#ffffff", font=dict(family=ECONOMIST_FONT, color=text_color)),
    width=1200,
    height=700,
    legend=dict(itemclick=False, itemdoubleclick=False),
    shapes=[
        dict(
            type="line",
            x0=0,
            x1=0,
            y0=-0.08,
            y1=1.05,
            xref="x",
            yref="paper",
            line=dict(color=grid_color, width=1.4)
        ),
        dict(
            type="line",
            x0=underline_x0,
            x1=underline_x1,
            y0=underline_y,
            y1=underline_y,
            xref="paper",
            yref="paper",
            line=dict(color=highlight_color, width=2.5)
        ),
    ],
)

x_axis_max = (
    math.ceil((dumbbell_base.max().max() if not dumbbell_base.empty else 0) / 5) * 5 + 5
)
dumbbell_fig.update_layout(
    xaxis=dict(
        title=dict(text="Participación (%)", font=dict(family=ECONOMIST_FONT, size=14, color=text_color)),
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        showgrid=True,
        gridcolor=grid_color,
        griddash="dot",
        zeroline=False,
        ticksuffix="%",
        range=[0, max(40, x_axis_max)],
    ),
    yaxis=dict(
        title="",
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        ticklabelstandoff=14,
        showgrid=False,
        zeroline=False,
        categoryorder="array",
        categoryarray=FOCUS_REGIONS,
    ),
)

dumbbell_fig

In [43]:
los_rios_generators_plot = (
    los_rios_generators.copy().fillna("Sin información")
    .sort_values("toneladas", ascending=True)
)

los_rios_generators_plot["label"] = los_rios_generators_plot["toneladas"].apply(humanize_tonnes)

highlight_color = "#d62828"
neutral_color = "#0a9396"
background_color = "#f7f2ea"
grid_color = "#d9d3c5"
text_color = "#1e1b18"

highlight_company = (
    los_rios_generators_plot.iloc[-1]["razon_social"] if not los_rios_generators_plot.empty else None
)

max_value = los_rios_generators_plot["toneladas"].max() if not los_rios_generators_plot.empty else 0
x_padding = max_value * 0.10 if max_value else 1
label_offset = max(max_value * 0.12, 1.0) if max_value else 1.0


generators_fig = go.Figure()

for _, row in los_rios_generators_plot.iterrows():
    color = highlight_color if row["razon_social"] == highlight_company else neutral_color
    label_color = highlight_color if row["razon_social"] == highlight_company else text_color

    generators_fig.add_trace(
        go.Scatter(
            x=[0, row["toneladas"]],
            y=[row["razon_social"], row["razon_social"]],
            mode="lines",
            line=dict(color=color, width=2.2),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    generators_fig.add_trace(
        go.Scatter(
            x=[row["toneladas"]],
            y=[row["razon_social"]],
            mode="markers",
            marker=dict(color=color, size=12, line=dict(color="#ffffff", width=1.2)),
            hovertemplate=("<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<extra></extra>"),
            customdata=[[row["razon_social"], row["toneladas"]]],
            cliponaxis=False,
            showlegend=False,
        )
    )

    generators_fig.add_trace(
        go.Scatter(
            x=[row["toneladas"] + label_offset],
            y=[row["razon_social"]],
            mode="text",
            text=[row["label"]],
            textposition="middle left",
            textfont=dict(family=ECONOMIST_FONT, size=12, color=label_color),
            hoverinfo="skip",
            showlegend=False,
            cliponaxis=False,
        )
    )

underline_x0 = -0.40  # Extiende el subrayado hacia la izquierda
underline_x1 = 0.95   # Ajusta este valor para el final del subrayado
underline_y = 1.10    # Ajusta este valor para acercar o alejar la línea del título

generators_fig.update_layout(
    template="plotly_white",
    title=dict(
        text="Top generadores en Los Ríos",
        font=dict(family=ECONOMIST_FONT, size=24, color=text_color),
        x=0.05,
        y=0.96,
    ),
    font=dict(family=ECONOMIST_FONT, size=15, color=text_color),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    margin=dict(l=250, r=200, t=120, b=80),
    hoverlabel=dict(bgcolor="#ffffff", font=dict(family=ECONOMIST_FONT, color=text_color)),
    width=1200,
    height=700,
    shapes=[
        dict(
            type="line",
            x0=0,
            x1=0,
            y0=-0.08,
            y1=1.05,
            xref="x",
            yref="paper",
            line=dict(color="#c9c1b4", width=1.2)
        ),
        dict(
            type="line",
            x0=underline_x0,
            x1=underline_x1,
            y0=underline_y,
            y1=underline_y,
            xref="paper",
            yref="paper",
            line=dict(color=highlight_color, width=4)
        ),
    ],
)

generators_fig.update_layout(
    xaxis=dict(
        title=dict(text="Toneladas", font=dict(family=ECONOMIST_FONT, size=14, color=text_color)),
        showgrid=True,
        gridcolor=grid_color,
        griddash="dot",
        zeroline=False,
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        range=[0, max_value + x_padding + label_offset + (0.05 * max_value if max_value else 0)],
    ),
    yaxis=dict(
        title="",
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        showgrid=False,
        zeroline=False,
        categoryorder="array",
        categoryarray=los_rios_generators_plot["razon_social"].tolist(),
    ),
)

generators_fig

# Panorama de residuos peligrosos RNP 2023 en Chile

## Propósito

Este cuaderno resume los hallazgos clave del sistema de Residuos Peligrosos (RNP) en Chile durante 2023. La historia combina métricas agregadas, análisis territoriales y una mirada a los principales generadores para informar decisiones de política pública y gestión empresarial.

## Hoja de ruta analítica

1. Configurar ambiente y cargar datasets depurados.

2. Extraer indicadores ejecutivos para contextualizar el volumen de residuos.

3. Visualizar la geografía de generación y su mezcla de tratamientos.

4. Identificar actores privados líderes y patrones sectoriales.

5. Resumir implicancias estratégicas para economía circular y fiscalización.

In [37]:
from __future__ import annotations



from pathlib import Path

from typing import Iterable



import numpy as np

import pandas as pd

import plotly.express as px

import plotly.graph_objects as go

from IPython.display import Markdown



pd.set_option("display.max_rows", 20)

pd.set_option("display.max_columns", 40)

pd.set_option("display.float_format", "{:,.2f}".format)



ECONOMIST_FONT = "Georgia, serif"

ECONOMIST_COLORS = [

    "#005f73",

    "#0a9396",

    "#94d2bd",

    "#ee9b00",

    "#ca6702",

    "#bb3e03",

    "#ae2012",

    "#001219",

]



def find_project_root(start: Path | None = None) -> Path:

    start_path = Path(start or Path.cwd()).resolve()

    for candidate in [start_path, *start_path.parents]:

        if (candidate / "data").is_dir() and (candidate / "src").is_dir():

            return candidate

    raise FileNotFoundError("No se encontró la raíz del proyecto con carpetas 'data' y 'src'.")



def apply_economist_style(fig: go.Figure, *, font_size: int = 16) -> go.Figure:

    fig.update_layout(

        template="plotly_white",

        font=dict(family=ECONOMIST_FONT, color="#1e293b"),

        title=dict(font=dict(size=font_size + 2, family=ECONOMIST_FONT, color="#1e293b")),

        xaxis=dict(

            title=dict(font=dict(family=ECONOMIST_FONT, color="#1e293b")),

            tickfont=dict(family=ECONOMIST_FONT, color="#1e293b"),

            showgrid=True,

            gridcolor="#cbd5e1",

            zeroline=False,

        ),

        yaxis=dict(

            title=dict(font=dict(family=ECONOMIST_FONT, color="#1e293b")),

            tickfont=dict(family=ECONOMIST_FONT, color="#1e293b"),

            showgrid=True,

            gridcolor="#cbd5e1",

            zeroline=False,

        ),

        legend=dict(font=dict(family=ECONOMIST_FONT, color="#1e293b")),

        bargap=0.25,

        plot_bgcolor="#ffffff",

        paper_bgcolor="#ffffff",

        margin=dict(l=80, r=40, t=80, b=60),

    )

    fig.update_traces(marker_line_color="#0f172a", marker_line_width=0.4)

    return fig



def humanize_tonnes(value: float) -> str:

    if value >= 1_000_000:

        return f"{value / 1_000_000:.1f} millones t"

    if value >= 1_000:

        return f"{value / 1_000:.1f} mil t"

    return f"{value:,.0f} t"


In [38]:
PROJECT_ROOT = find_project_root()

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"



clean_df = pd.read_parquet(PROCESSED_DIR / "rinp_generacion_2023_cl_clean.parquet")



region_df = pd.read_csv(PROCESSED_DIR / "region_summary.csv", sep=";")

region_df.columns = ["region", "toneladas_sum", "toneladas_promedio", "registros"]

region_df["toneladas_sum"] = pd.to_numeric(region_df["toneladas_sum"], errors="coerce")

region_df["toneladas_promedio"] = pd.to_numeric(region_df["toneladas_promedio"], errors="coerce")

region_df["registros"] = pd.to_numeric(region_df["registros"], errors="coerce", downcast="integer")

region_df = region_df.sort_values("toneladas_sum", ascending=False).reset_index(drop=True)



FOCUS_REGIONS = [

    "Los Ríos",

    "Biobío",

    "Metropolitana De Santiago",

    "Los Lagos",

    "Araucanía",

]

focus_region_df = region_df[region_df["region"].isin(FOCUS_REGIONS)].copy()

focus_region_df["region"] = pd.Categorical(

    focus_region_df["region"], categories=FOCUS_REGIONS, ordered=True

)

focus_region_df = focus_region_df.sort_values("toneladas_sum", ascending=False).reset_index(drop=True)



treatment_df = pd.read_csv(PROCESSED_DIR / "region_treatment.csv", sep=";")

treatment_df.columns = ["region", "tratamiento_nivel_1", "toneladas"]

treatment_df["toneladas"] = pd.to_numeric(treatment_df["toneladas"], errors="coerce")

treatment_df = treatment_df.sort_values("toneladas", ascending=False).reset_index(drop=True)



overall_mix = (

    treatment_df.groupby("tratamiento_nivel_1", as_index=False)["toneladas"].sum()

    .sort_values("toneladas", ascending=False)

    .reset_index(drop=True)

)

overall_mix["share"] = overall_mix["toneladas"] / overall_mix["toneladas"].sum()



treatment_share_df = treatment_df.copy()

treatment_share_df["share"] = treatment_share_df["toneladas"] / treatment_share_df.groupby("region")["toneladas"].transform("sum")



focus_treatment_share_df = treatment_share_df[

    treatment_share_df["region"].isin(FOCUS_REGIONS)

].copy()

focus_treatment_share_df["region"] = pd.Categorical(

    focus_treatment_share_df["region"], categories=FOCUS_REGIONS, ordered=True

)



top_generators = (

    clean_df.groupby("razon_social", dropna=False)["cantidad_toneladas"].sum()

    .sort_values(ascending=False)

    .head(15)

    .reset_index()

    .rename(columns={"cantidad_toneladas": "toneladas"})

)

top_generators["toneladas"] = top_generators["toneladas"].astype(float)



los_rios_generators = (

    clean_df[clean_df["region"] == "Los Ríos"]

    .groupby("razon_social", dropna=False)["cantidad_toneladas"].sum()

    .sort_values(ascending=False)

    .head(10)

    .reset_index()

    .rename(columns={"cantidad_toneladas": "toneladas"})

)

los_rios_generators["toneladas"] = los_rios_generators["toneladas"].astype(float)



rubro_df = (

    clean_df.groupby("rubro_vu", dropna=False)["cantidad_toneladas"].sum()

    .sort_values(ascending=False)

    .head(10)

    .reset_index()

    .rename(columns={"cantidad_toneladas": "toneladas"})

)



los_rios_rubro_df = (

    clean_df[clean_df["region"] == "Los Ríos"]

    .groupby("rubro_vu", dropna=False)["cantidad_toneladas"].sum()

    .sort_values(ascending=False)

    .head(10)

    .reset_index()

    .rename(columns={"cantidad_toneladas": "toneladas"})

)



los_rios_total = focus_region_df.loc[

    focus_region_df["region"] == "Los Ríos", "toneladas_sum"

].sum()

los_rios_share = los_rios_total / region_df["toneladas_sum"].sum()



headline_metrics = pd.DataFrame(

    {

        "Indicador": [

            "Residuos totales registrados",

            "Regiones con reportes",

            "Establecimientos únicos",

            "Tratamiento predominante",

        ],

        "Valor": [

            humanize_tonnes(clean_df["cantidad_toneladas"].sum()),

            clean_df["region"].nunique(),

            clean_df["nombre_establecimiento"].nunique(),

            overall_mix.iloc[0]["tratamiento_nivel_1"],

        ],

    }

)



headline_metrics


,Indicador,Valor
0,Residuos totales registrados,10.4 millones t
1,Regiones con reportes,16
2,Establecimientos únicos,8411
3,Tratamiento predominante,Eliminación


In [ ]:
total_tonnes = humanize_tonnes(clean_df["cantidad_toneladas"].sum())

region_count = clean_df["region"].nunique()

facility_count = clean_df["nombre_establecimiento"].nunique()

top_treatment = overall_mix.iloc[0]["tratamiento_nivel_1"]

los_rios_share_pct = los_rios_share * 100



Markdown(

    f"""

### Lectura ejecutiva



- Chile gestionó más de **{total_tonnes}** en 2023, con cobertura efectiva en **{region_count}** regiones y **{facility_count}** establecimientos activos.



- **Los Ríos explica el {los_rios_share_pct:.1f}%** del total nacional dentro del bloque comparativo que incluye Biobío, Metropolitana, Los Lagos y Araucanía.



- El tratamiento dominante fue **{top_treatment}**, lo que evidencia espacio para acelerar modelos de valorización.



- A continuación se examinan patrones geográficos, mix de tratamientos y actores líderes que explican el panorama nacional con énfasis en Los Ríos.

"""

)



### Lectura ejecutiva



- Chile gestionó más de **10.4 millones t** en 2023, con cobertura efectiva en **16** regiones y **8411** establecimientos activos.



- **Los Ríos explica el 2.3%** del total nacional dentro del bloque comparativo que incluye Biobío, Metropolitana, Los Lagos y Araucanía.



- El tratamiento dominante fue **Eliminación**, lo que evidencia espacio para acelerar modelos de valorización.



- A continuación se examinan patrones geográficos, mix de tratamientos y actores líderes que explican el panorama nacional con énfasis en Los Ríos.



## Indicadores clave



Los siguientes indicadores cuantifican la magnitud y cobertura del sistema RNP 2023, sirviendo como punto de partida para el análisis territorial y sectorial.

In [ ]:
zebra_fill = ["#ffffff" if idx % 2 == 0 else "#f1f5f9" for idx in range(len(headline_metrics))]



headline_table = go.Figure(

    data=[

        go.Table(

            columnorder=[1, 2],

            header=dict(

                values=["Indicador", "Valor"],

                fill_color="#005f73",

                font=dict(color="white", family=ECONOMIST_FONT, size=16),

                align="left",

                height=36,

            ),

            cells=dict(

                values=[headline_metrics["Indicador"].tolist(), headline_metrics["Valor"].tolist()],

                fill_color=[zebra_fill, zebra_fill],

                font=dict(color="#1e293b", family=ECONOMIST_FONT, size=15),

                align="left",

                height=30,

            ),

        )

    ]

)

headline_table.update_layout(margin=dict(l=0, r=0, t=10, b=0), height=240)

headline_table

## Panorama nacional



Antes de contrastar las regiones foco, revisamos la distribución global para identificar el peso relativo de cada territorio dentro del sistema RNP 2023.

In [ ]:
national_region_plot = (
    region_df.copy().sort_values("toneladas_sum", ascending=True).reset_index(drop=True)
)

national_region_plot["label"] = national_region_plot["toneladas_sum"].apply(humanize_tonnes)

highlight_region = "Los Ríos" if "Los Ríos" in national_region_plot["region"].values else None
highlight_color = "#d62828"
neutral_color = "#0a9396"

national_fig = go.Figure()

for _, row in national_region_plot.iterrows():
    color = highlight_color if row["region"] == highlight_region else neutral_color
    national_fig.add_trace(
        go.Scatter(
            x=[0, row["toneladas_sum"]],
            y=[row["region"], row["region"]],
            mode="lines",
            line=dict(color=color, width=2.2),
            hoverinfo="skip",
            showlegend=False,
        )
    )
    national_fig.add_trace(
        go.Scatter(
            x=[row["toneladas_sum"]],
            y=[row["region"]],
            mode="markers+text",
            marker=dict(color=color, size=12, line=dict(color="#ffffff", width=1.2)),
            text=[row["label"]],
            textposition="middle right",
            textfont=dict(family=ECONOMIST_FONT, size=12, color="#1e1b18"),
            hovertemplate=("<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<extra></extra>"),
            customdata=[[row["region"], row["toneladas_sum"]]],
            cliponaxis=False,
            showlegend=False,
        )
    )

background_color = "#f7f2ea"
grid_color = "#d9d3c5"
text_color = "#1e1b18"

max_value = national_region_plot["toneladas_sum"].max()
x_padding = max_value * 0.1 if max_value else 1

underline_x0 = -0.30  # Extiende el subrayado hacia la izquierda
underline_x1 = 0.82   # Ajusta este valor para el final del subrayado
underline_y = 1.08    # Ajusta este valor para acercar o alejar la línea del título

national_fig.update_layout(
    template="plotly_white",
    title=dict(
        text="Toneladas declaradas por región",
        font=dict(family=ECONOMIST_FONT, size=24, color=text_color),
        x=0.02,
        y=0.94,
    ),
    font=dict(family=ECONOMIST_FONT, size=15, color=text_color),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    margin=dict(l=180, r=140, t=120, b=60),
    hoverlabel=dict(bgcolor="#ffffff", font=dict(family=ECONOMIST_FONT, color=text_color)),
    width=1200,
    height=720,
    shapes=[
        dict(
            type="line",
            x0=0,
            x1=0,
            y0=-0.08,
            y1=1.05,
            xref="x",
            yref="paper",
            line=dict(color="#c9c1b4", width=1.2)
        ),
        dict(
            type="line",
            x0=underline_x0,
            x1=underline_x1,
            y0=underline_y,
            y1=underline_y,
            xref="paper",
            yref="paper",
            line=dict(color=highlight_color, width=4)
        ),
    ],
)

national_fig.update_layout(
    xaxis=dict(
        title=dict(text="Toneladas", font=dict(family=ECONOMIST_FONT, size=14, color=text_color)),
        showgrid=True,
        gridcolor=grid_color,
        griddash="dot",
        zeroline=False,
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        range=[0, max_value + x_padding],
    ),
    yaxis=dict(
        title="",
        tickfont=dict(family=ECONOMIST_FONT, size=12, color=text_color),
        showgrid=False,
        zeroline=False,
        categoryorder="array",
        categoryarray=national_region_plot["region"].tolist(),
    ),
)

national_fig.update_traces(textfont=dict(family=ECONOMIST_FONT, size=12, color=text_color))

national_fig

In [ ]:
top_national_regions = national_region_plot.sort_values("toneladas_sum", ascending=False).reset_index(drop=True)

los_rios_rank = (

    top_national_regions.index[top_national_regions["region"] == "Los Ríos"].tolist()[0] + 1

)

los_rios_value = top_national_regions.loc[

    top_national_regions["region"] == "Los Ríos", "toneladas_sum"

].iloc[0]



Markdown(

    f"""

Metropolitana lidera con **{humanize_tonnes(top_national_regions.iloc[0]['toneladas_sum'])}**, seguida por Biobío (**{humanize_tonnes(top_national_regions.iloc[1]['toneladas_sum'])}**).

Los Ríos se ubica en el lugar {los_rios_rank} del ranking nacional, con **{humanize_tonnes(los_rios_value)}**.

"""

)



Metropolitana lidera con **2.9 millones t**, seguida por Biobío (**1.7 millones t**).

Los Ríos se ubica en el lugar 11 del ranking nacional, con **241.9 mil t**.



## Geografía de la generación



Contrastamos las cinco regiones prioritarias (Los Ríos, Biobío, Metropolitana, Los Lagos y Araucanía) para dimensionar la posición relativa de Los Ríos dentro del bloque sur-centro.

In [ ]:
focus_region_stats = (

    focus_region_df.set_index("region")["toneladas_sum"].apply(humanize_tonnes)

)



Markdown(

    f"""

Los Ríos declara **{focus_region_stats['Los Ríos']}**, muy por debajo de Biobío (**{focus_region_stats['Biobío']}**) y de Metropolitana (**{focus_region_stats['Metropolitana De Santiago']}**),

pero sigue siendo un actor regional relevante cuyo crecimiento podría cerrar parcialmente la brecha con Araucanía (**{focus_region_stats['Araucanía']}**) y Los Lagos (**{focus_region_stats['Los Lagos']}**).

"""

)



Los Ríos declara **241.9 mil t**, muy por debajo de Biobío (**1.3 millones t**) y de Metropolitana (**2.9 millones t**),

pero sigue siendo un actor regional relevante cuyo crecimiento podría cerrar parcialmente la brecha con Araucanía (**615.3 mil t**) y Los Lagos (**747.7 mil t**).



## Mezcla de tratamientos



El bloque comparativo permite evaluar si Los Ríos mantiene un patrón de gestión similar a Biobío, Metropolitana, Los Lagos y Araucanía o si abre espacio para estrategias diferenciadas.

In [49]:
overall_mix_plot = overall_mix.copy().sort_values("share", ascending=True)

overall_mix_plot["share_pct"] = overall_mix_plot["share"] * 100

overall_mix_plot["label"] = overall_mix_plot["share_pct"].map(lambda v: f"{v:.1f}%")



overall_mix_fig = px.bar(

    overall_mix_plot,

    x="share_pct",

    y="tratamiento_nivel_1",

    orientation="h",

    text="label",

    color="tratamiento_nivel_1",

    color_discrete_sequence=ECONOMIST_COLORS,

    title="Participación nacional por tipo de tratamiento",

)

overall_mix_fig = apply_economist_style(overall_mix_fig)

overall_mix_fig.update_traces(textposition="outside")

overall_mix_fig.update_layout(

    showlegend=False,

    xaxis_title="Participación (%)",

    yaxis_title="",

    xaxis_ticksuffix="%",

    xaxis_range=[0, max(35, overall_mix_plot["share_pct"].max() + 5)],

)

overall_mix_fig

In [62]:
import math

rubro_plot = (
    los_rios_rubro_df.copy()
    .fillna("Sin clasificar")
    .sort_values("toneladas", ascending=True)
    )

if rubro_plot.empty:
    rubro_fig = go.Figure()
    rubro_fig.update_layout(title="Principales rubros en Los Ríos por volumen generado")
    rubro_fig.show()
else:
    rubro_plot["tonnes_millions"] = rubro_plot["toneladas"] / 1_000_000
    rubro_plot["share"] = rubro_plot["toneladas"] / rubro_plot["toneladas"].sum()

    def format_spanish(number: float) -> str:
        formatted = f"{number:,.1f}"
        return formatted.replace(",", "X").replace(".", ",").replace("X", ".")

    rubro_plot["label"] = rubro_plot.apply(
        lambda row: f"{format_spanish(row['tonnes_millions'])} M t · {format_spanish(row['share'] * 100)}%",
        axis=1,
    )

    highlight_color = "#d62828"
    neutral_color = "#9ca3af"
    highlight_category = rubro_plot.iloc[-1]["rubro_vu"]

    rubro_fig = go.Figure()

    for _, row in rubro_plot.iterrows():
        color = highlight_color if row["rubro_vu"] == highlight_category else neutral_color

        rubro_fig.add_trace(
            go.Scatter(
                x=[0, row["tonnes_millions"]],
                y=[row["rubro_vu"], row["rubro_vu"]],
                mode="lines",
                line=dict(color=color, width=3),
                hoverinfo="skip",
                showlegend=False,
            )
        )

        rubro_fig.add_trace(
            go.Scatter(
                x=[row["tonnes_millions"]],
                y=[row["rubro_vu"]],
                mode="markers+text",
                marker=dict(color=color, size=14, line=dict(color="#ffffff", width=1.5)),
                text=[row["label"]],
                textposition="middle right",
                textfont=dict(family="Georgia, serif", color="#1e1b18", size=12),
                hovertemplate=(
                    "<b>%{customdata[0]}</b><br>Toneladas: %{customdata[1]:,.0f}<br>Participación regional: %{customdata[2]:.1%}<extra></extra>"
                ),
                customdata=[[row["rubro_vu"], row["toneladas"], row["share"]]],
                showlegend=False,
            )
        )

    text_color = "#1e1b18"
    background_color = "#f7f2ea"
    grid_color = "#d9d3c5"

    rubro_fig.update_layout(
        template="plotly_white",
        font=dict(family="Georgia, serif", color=text_color, size=15),
        title=dict(text="Principales rubros en Los Ríos por volumen generado", font=dict(size=24, color=text_color), x=0.05, y=0.94),
        xaxis=dict(
            title=dict(text="Toneladas", font=dict(family="Georgia, serif", color=text_color, size=14)),
            tickfont=dict(family="Georgia, serif", color=text_color, size=12),
            showgrid=True,
            gridcolor=grid_color,
            griddash="dot",
            zeroline=False,
            linecolor=grid_color,
        ),
        yaxis=dict(
            title="",
            tickfont=dict(family="Georgia, serif", color=text_color, size=12),
            showgrid=False,
            zeroline=False,
            linecolor=grid_color,
        ),
        plot_bgcolor=background_color,
        paper_bgcolor=background_color,
        hoverlabel=dict(bgcolor="#ffffff", font=dict(family="Georgia, serif", color=text_color)),
        margin=dict(l=200, r=170, t=140, b=130),
        width=1380,
        height=760,
    )

    underline_start = -0.26  # move left/right
    underline_end = 0.84    # adjust length towards right
    underline_height = 1.12  # raise/lower relative to title

    underline_shape = dict(
        type="line",
        x0=underline_start,
        x1=underline_end,
        y0=underline_height,
        y1=underline_height,
        xref="paper",
        yref="paper",
        line=dict(color=highlight_color, width=4),
    )
    existing_shapes = list(rubro_fig.layout.shapes) if rubro_fig.layout.shapes else []
    rubro_fig.update_layout(shapes=[underline_shape, *existing_shapes])

    max_millions = rubro_plot["tonnes_millions"].max()
    tick_max = math.ceil(max_millions * 10) / 10 if max_millions > 0 else 0.5
    tick_step = max(0.2, tick_max / 6)
    tick_values = [round(i * tick_step, 1) for i in range(int(tick_max / tick_step) + 1)]
    if tick_values[-1] < tick_max:
        tick_values.append(round(tick_max, 1))
    tick_values = sorted(set(tick_values))
    tick_text = [f"{format_spanish(val)} M" for val in tick_values]

    x_upper_padding = tick_step if tick_step > 0 else 0.2
    xaxis_range = [0, tick_max + x_upper_padding]

    rubro_fig.update_traces(textfont=dict(size=12, family="Georgia, serif"))

    rubro_fig.update_layout(
        xaxis=dict(
            tickvals=tick_values,
            ticktext=tick_text,
            rangemode="tozero",
            range=xaxis_range,
        ),
        yaxis=dict(
            categoryorder="array",
            categoryarray=rubro_plot["rubro_vu"].tolist(),
        ),
        annotations=[
            dict(
                text="Los Ríos, RNP 2023",
                x=0,
                y=1.06,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(family="Georgia, serif", size=14, color="#4a4338"),
            ),
            dict(
                text="Fuente: Registro Nacional de Emisiones y Transferencias de Residuos (RNP) 2023",
                x=0,
                y=-0.22,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(family="Georgia, serif", size=11, color="#6e6252"),
            ),
        ],
    )

    rubro_fig.show()

## Actores líderes y rubros dominantes



Centramos la lupa en los generadores y rubros de Los Ríos para detectar empresas ancla y cadenas productivas que explican su peso relativo frente al resto del bloque comparativo.

In [ ]:
los_rios_share_pct = los_rios_share * 100

national_rank_df = (

    region_df.copy()

    .sort_values("toneladas_sum", ascending=False)

    .reset_index(drop=True)

)

los_rios_rank = (

    national_rank_df.index[national_rank_df["region"] == "Los Ríos"].tolist()[0] + 1

)



Markdown(

    f"""

## Cierre estratégico



- Los Ríos concentra aproximadamente **{los_rios_share_pct:.1f}%** del bloque sur-centro comparado (Biobío, Metropolitana, Los Lagos y Araucanía), situándose por detrás de los polos más industrializados pero ofreciendo margen de crecimiento controlado.

- En el panorama nacional ocupa el **puesto {los_rios_rank}** en volumen declarado, lo que refuerza la necesidad de políticas focalizadas para escalar su contribución.

- El mix local combina valorización y eliminación en proporciones similares a Biobío, lo que sugiere priorizar inversiones en valorización y captura energética para mover la aguja.

- Las empresas y rubros líderes de la región ofrecen un punto de entrada para pactos de economía circular y fiscalización diferenciada.

"""

)



## Cierre estratégico



- Los Ríos concentra aproximadamente **2.3%** del bloque sur-centro comparado (Biobío, Metropolitana, Los Lagos y Araucanía), situándose por detrás de los polos más industrializados pero ofreciendo margen de crecimiento controlado.

- En el panorama nacional ocupa el **puesto 11** en volumen declarado, lo que refuerza la necesidad de políticas focalizadas para escalar su contribución.

- El mix local combina valorización y eliminación en proporciones similares a Biobío, lo que sugiere priorizar inversiones en valorización y captura energética para mover la aguja.

- Las empresas y rubros líderes de la región ofrecen un punto de entrada para pactos de economía circular y fiscalización diferenciada.

